# Imports

In [ ]:
import warnings
from functools import partial
from importlib import reload
from operator import getitem

import dask
import holoviews as hv
import matplotlib.pyplot as plt
import nd2reader
import numpy as np
import pandas as pd
import streamz
import zarr
from bokeh.models.tools import HoverTool
from cytoolz import compose
from dask import delayed
from dask.distributed import Client, LocalCluster
from dask_jobqueue import SLURMCluster
from holoviews.streams import Stream, param
from tqdm import tnrange, tqdm, tqdm_notebook

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import trench_detection
import util

In [ ]:
%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0

# Config

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="1:00:00",
    # job_extra=['-p transfer'],
    # job_extra=['--cores-per-socket=8'],
    # interface='ib0',
    memory="64GB",
    # local_directory='/tmp',
    threads=1,
    processes=1,
    diagnostics_port=("127.0.0.1", 8787),
)
client = Client(cluster)

In [ ]:
cluster._widget().children[1].children[1].children[0].children[0].layout.width = "200px"
cluster

In [ ]:
cluster = LocalCluster(n_workers=0, ncores=1, processes=False)  # TODO: ncores ignored
cluster.start_worker(ncores=1)
client = Client(cluster)

In [ ]:
client

# Functions

# Loading data

In [ ]:
nd2_filenames = [
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr.nd2",
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr001.nd2",
]

In [ ]:
nd2s = {
    filename: client.submit(nd2reader.ND2Reader, filename, memmap=False)
    for filename in nd2_filenames
}

In [ ]:
nd2 = nd2s[nd2_filenames[0]]

In [ ]:
a = client.submit(
    compose(
        trench_detection.get_trenches, lambda x: x.get_frame_2D(v=2, c=2, memmap=False)
    ),
    nd2,
)

In [ ]:
a = client.submit(
    compose(
        util.wrap_diagnostics(trench_detection.get_trenches),
        lambda x: x.get_frame_2D(v=0, c=2, memmap=False),
    ),
    nd2,
)

In [ ]:
a.result()

In [ ]:
b = util.future_getitem(client.submit, a, 1)

In [ ]:
b

In [ ]:
b.result()

In [ ]:
def do_reload():
    from importlib import reload

    reload(trench_detection)


client.run(do_reload)

In [ ]:
reload(trench_detection)

In [ ]:
a.result()

In [ ]:
trench_data = {
    filename: {
        v: client.submit(
            compose(
                util.wrap_diagnostics(trench_detection.get_trenches),
                lambda x: x.get_frame_2D(v=v, c=2, memmap=False),
            ),
            nd2,
        )
        for v in range(2)
    }
    for filename, nd2 in nd2s.items()
}

In [ ]:
trench_data

In [ ]:
import traceback
from traceback import print_exception, print_tb

In [ ]:
traceback.extract_tb(t)

# Scratch

In [ ]:
def load_nd2(filename):
    nd2 = nd2reader.ND2Reader(filename)
    diag = tree()
    return get_trenches(nd2[1, 30], diagnostics=diag_pos[pos])

In [ ]:
trench_positions = {}

# Old

In [ ]:
get_trenches(root_group["raw"][str(pos)][1, 30], diagnostics=diag_pos[pos])

In [ ]:
diag = tree()
_ = get_trenches(root_group["raw"][str(pos)][0, 1], diagnostics=diag)

In [ ]:
def f(img_stack):
    ary = np.stack(
        [
            segment_trench(img_stack[t], diagnostics=None)
            for t in range(img_stack.shape[0])
        ],
        axis=0,
    )
    ary = zarr.array(ary, compressor=DEFAULT_FRAME_COMPRESSOR)
    return ary


trench_seg_masks = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=1,
    preload=True,
    time_slice=slice(None),
    positions=range(1),
)

In [ ]:
def f(img_stack):
    return pd.Series(np.percentile(img_stack, 95, axis=(1, 2)))
    # return pd.Series(np.max(img_stack, axis=(1,2)))


trench_traces_all = positionwise_trenchwise_map(
    root_group["raw"],
    trench_points_pos,
    f,
    channel_slice=2,
    preload=True,
    time_slice=slice(None),
    positions=range(100),
)